In [1]:
import json
import pandas as pd

In [2]:
fieldsOfStudy = set()

In [3]:
with open('dblp_papers_v11.txt', 'r') as file:
    for line in file:
        data = json.loads(line)
        for fos in data.get('fos',[]):
            fieldsOfStudy.add(fos.get('name',''))

In [ ]:
data = [json.loads(line) for line in open('dblp_papers_v11.txt', 'r')]

In [4]:
fieldsOfStudy

{'Experiment Object',
 'Light cone',
 'Curved corridor',
 'LAPDm',
 'Symmetry protected topological order',
 'Delusion',
 'Group intelligence',
 'IEC 62700',
 'Polyene',
 'Fetal skin',
 'Drag reducing agent',
 'Transplanted tissue',
 'Chain of events (aeronautics)',
 'Alnus incana',
 'Complete group',
 'Optical burst switching',
 'Collision response',
 'Aliivibrio fischeri',
 'Illusory correlation',
 'SNOBOL',
 'Solar cell research',
 'Hall effect thruster',
 'Sternal recumbency',
 'Primary hyperparathyroidism',
 'Macaronic language',
 'Search cost',
 'Length Fraction',
 'Algebra of physical space',
 'Equity (finance)',
 'Terminology extraction',
 'Four-way valve',
 'Early Therapy',
 'Participant Object',
 'Arithmetic overflow',
 'Converters',
 'Superstatistics',
 'Planck energy',
 'Stereochemistry',
 'NETtalk',
 'SNePS',
 'Chicken manure',
 'Dental Procedure',
 'Predicable',
 'Thermocouple',
 'Bark scale',
 'Expertise finding',
 'Minimum Data Set',
 'Spring break',
 'Zaocys dhumnades'

In [20]:
data

{'id': '99999932',
 'title': 'IIT Kharagpur at TAC 2008: Statistical Model for Opinion Summarization',
 'authors': [{'name': 'Sushant Kumar',
   'id': '2590270817',
   'org': 'Department of Computer Science and Engineering|Indian Institute of Technology Kharagpur'},
  {'name': 'Diptesh Chatterjee',
   'id': '2152412452',
   'org': 'Department of Computer Science and Engineering|Indian Institute of Technology Kharagpur'}],
 'venue': {'raw': 'Theory and Applications of Categories', 'id': '2501765825'},
 'year': 2008,
 'n_citation': 0,
 'page_start': '',
 'page_end': '',
 'doc_type': 'Journal',
 'publisher': '',
 'volume': '',
 'issue': '',
 'references': ['2110693578', '2163455955', '2166706824'],
 'indexed_abstract': {'IndexLength': 44,
  'InvertedIndex': {'In': [0],
   'this': [1],
   'paper': [2],
   'we': [3],
   'present': [4, 33],
   'our': [5],
   'participation': [6],
   'at': [7],
   'TAC': [8, 40],
   '2008': [9, 41],
   'opinion': [10, 21],
   'summarization': [11, 26],
   'ta

In [12]:
type(data['references'][0])

str

In [34]:
with open('dblp_papers_v11.txt', 'r') as file:
    for line in file:
        data = json.loads(line)
        if 'indexed_abstract' in data.keys():
            print(data['indexed_abstract'])
            break

{'IndexLength': 173, 'InvertedIndex': {'Both': [0], 'in': [1, 64], 'the': [2, 43, 70, 93, 101, 110, 118, 124, 129, 133, 136, 140, 142, 151, 166], 'cloud': [3], 'and': [4, 18, 24, 77, 126, 138, 156, 171], 'mobile': [5], 'environments,': [6], 'a': [7, 36, 83, 157], 'large': [8], 'number': [9], 'of': [10, 51, 95, 128, 135, 139], 'online': [11], 'services': [12], 'is': [13, 114, 168], 'daily': [14], 'accessed': [15], 'through': [16], 'smartphones': [17, 163], 'tablets.': [19], 'Since': [20], 'several': [21], 'security,': [22], 'safety': [23], 'trust': [25], 'concerns': [26], 'may': [27, 34], 'arise': [28], 'when': [29, 68], 'using': [30, 69], 'these': [31, 47], 'services,': [32], 'providers': [33, 56, 88], 'require': [35], 'usage': [37], 'policy': [38, 52, 120], 'to': [39, 57, 74, 89, 121, 148], 'be': [40], 'enforced': [41], 'on': [42, 100], 'devices': [44, 62], 'while': [45], 'accessing': [46], 'services.': [48], 'This': [49], 'kind': [50], 'enforcements': [53], 'enables': [54], 'service'

In [31]:
data.keys()

dict_keys(['id', 'title', 'authors', 'venue', 'year', 'n_citation', 'page_start', 'page_end', 'doc_type', 'publisher', 'volume', 'issue', 'references', 'indexed_abstract', 'fos'])

In [33]:
'indexed_abstract' in data.keys()

True

In [3]:
AITopLevelTopics = ['Artificial intelligence', 'Computer vision', 'Data mining',
                     'Data science', 'Machine learning', 'Natural language processing',
                     'Pattern recognition', 'Speech recognition']

In [8]:
for topic in AITopLevelTopics:
    if topic not in fieldsOfStudy:
        print('Not found topic: ', topic)

In [13]:
count = 0
with open('dblp_papers_v11.txt', 'r') as file:
    for line in file:
        if (count < 2):
            count += 1
            continue
        data = json.loads(line)
        break
            

In [14]:
data

{'id': '1000022707',
 'title': 'A SIMPLE OBSERVATION REGARDING ITERATIONS OF FINITE-VALUED POLYNOMIAL-TIME FUNCTIONS',
 'authors': [{'name': 'Jerzy Mycka', 'id': '263067851'}],
 'venue': {'raw': 'Reports on Mathematical Logic', 'id': '42464861'},
 'year': 2009,
 'n_citation': 0,
 'page_start': '19',
 'page_end': '29',
 'doc_type': 'Journal',
 'publisher': '',
 'volume': '44',
 'issue': '',
 'references': ['1972178849', '2069792094'],
 'indexed_abstract': {'IndexLength': 49,
  'InvertedIndex': {'A': [0],
   'b': [1],
   's': [2],
   't': [3],
   'r': [4],
   'a': [5, 31],
   'c': [6],
   't.': [7],
   'We': [8],
   'present': [9],
   'this': [10],
   'note': [11],
   'to': [12, 25],
   'point': [13],
   'out': [14],
   'that': [15],
   'the': [16, 43],
   'finitevalued': [17],
   'polynomial-time': [18],
   'computable': [19, 47],
   'functions': [20],
   'are': [21],
   'closed': [22],
   'with': [23],
   'respect': [24],
   'iteration.': [26],
   'This': [27],
   'fact': [28],
   'is'

In [17]:
abstract = [w for w in data['indexed_abstract']['InvertedIndex'].keys() if len(w) > 1]

In [19]:
AITopLevelTopicsSet = set(AITopLevelTopics)
papersUnderConsideration = set()
with open('dblp_papers_v11.txt', 'r') as file:
    for line in file:
        data = json.loads(line)
        for fos in data.get('fos',[]):
            if fos.get('name','') in AITopLevelTopicsSet:
                papersUnderConsideration.add(data['id'])

In [4]:
# paperList = list(papersUnderConsideration)
# with open("dblpPaperIDs.json", 'w') as f:
#     json.dump(paperList, f)
# To Read
with open("dblpPaperIDs.json", 'r') as f:
    paperList = json.load(f)
papersUnderConsideration = set(paperList)

In [18]:
paperData = dict()
with open('dblp_papers_v11.txt', 'r') as file:
    with open('dblp_AIpapers_v11.json', 'w') as outfile:
        for line in file:
            data = json.loads(line)
            paperID = data.get('id','')
            if paperID in papersUnderConsideration:
                dataDict = dict()
                dataDict['title'] = data.get('title', '')
                references = list()
                for reference in data.get('references',[]):
                    if reference in papersUnderConsideration:
                        references.append(reference)
                dataDict['references'] = references
                dataDict['abstract'] = []
                if 'indexed_abstract' in data:
                    dataDict['abstract'] = [w for w in data['indexed_abstract']['InvertedIndex'].keys() if len(w) > 1]                    
                dataDict['fos'] = data.get('fos',[])
                # paperData[paperID] = dataDict
                tmpDict = dataDict.copy()
                tmpDict['id'] = paperID
                json.dump(tmpDict, outfile)
                outfile.write('\n')

In [13]:
data1 = {"example1":1, "example2":2}
data2 = {"example21":21, "example22":22}

In [14]:
with open("saveData.json", "w") as outfile:
    json.dump(data1, outfile)
    outfile.write('\n')
    json.dump(data2, outfile)
    outfile.write('\n')

In [15]:
with open("saveData.json", "r") as infile:
    data = []
    for line in infile:
        data.append(json.loads(line))

In [16]:
data[0].keys()

dict_keys(['example1', 'example2'])

In [17]:
data

[{'example1': 1, 'example2': 2}, {'example21': 21, 'example22': 22}]

In [20]:
with open('dblp_AIpapers_v11.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        break

In [22]:
'1578000111' in papersUnderConsideration

True